In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
Rootdf = pd.read_csv('Root_Insurance_data_Edited.csv')


Rootdf['Multi_VD'] = Rootdf.Num_Vehicles * Rootdf.Num_Drivers
Rootdf['Add_VD'] = Rootdf.Num_Vehicles + Rootdf.Num_Drivers

Rootdf = Rootdf.replace('Y',1)
Rootdf = Rootdf.replace('N',0)
Rootdf = Rootdf.replace('U',2)
Rootdf = Rootdf.replace('S',0)
Rootdf = Rootdf.replace('M',1)


#Rootdf.Insured[Rootdf.Insured == 'N'] = 0
#Rootdf.Insured[Rootdf.Insured == 'Y'] = 1
#Rootdf.Insured[Rootdf.Insured == 'U'] = 2

#Rootdf.Marital[Rootdf.Marital == 'S'] = 0
#Rootdf.Marital[Rootdf.Marital == 'M'] = 1

#Rootdf.Click[Rootdf.Click == 'N'] = 0
#Rootdf.Click[Rootdf.Click == 'Y'] = 1

In [4]:
Rootdf.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Bid,Rank,Click,Policies_sold,Multi_VD,Add_VD
0,2,2,1,1,10,2,0,0,2,3
1,1,3,1,1,10,5,0,0,3,4
2,1,3,2,1,10,5,0,0,6,5
3,0,3,2,0,10,4,0,0,6,5
4,2,2,2,0,10,2,0,0,4,4


In [5]:
X = Rootdf[["Insured", "Num_Vehicles", "Num_Drivers", "Marital","Multi_VD","Add_VD"]]
Y = Rootdf.Rank
X.head()

,Insured,Num_Vehicles,Num_Drivers,Marital,Multi_VD,Add_VD
0,2,2,1,1,2,3
1,1,3,1,1,3,4
2,1,3,2,1,6,5
3,0,3,2,0,6,5
4,2,2,2,0,4,4


In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,
                                                    random_state = 314,
                                                    shuffle=True,
                                                    stratify=Rootdf[['Rank','Click','Policies_sold']])

In [7]:
#print(x_train.Rank.value_counts()/x_train.Rank.count())
#print(x_test.Rank.value_counts()/x_test.Rank.count())

#print(x_train.Click.value_counts()/x_train.Click.count())
#print(x_test.Click.value_counts()/x_test.Click.count())

print(y_train.value_counts()/y_train.count())
print(y_test.value_counts()/y_test.count())

3    0.240125
5    0.229125
4    0.208875
1    0.161000
2    0.160875
Name: Rank, dtype: float64
3    0.2400
5    0.2290
4    0.2090
1    0.1615
2    0.1605
Name: Rank, dtype: float64


In [8]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

def powerset(s):
    power_set = []
    x = len(s)
    for i in range(1 << x):
        power_set.append([s[j] for j in range(x) if (i & (1 << j))])
        
    return power_set[1:]

In [20]:
## get all models we're interested in
models = powerset(["Insured", "Num_Vehicles", "Num_Drivers", "Marital","Multi_VD","Add_VD"])
models.append("baseline")

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 5309)

## make empty mse holder
cv_roc_auc_ovo_model = np.zeros((5, len(models)))
cv_roc_auc_ovr_model = np.zeros((5, len(models)))
cv_acc_score_model = np.zeros((5, len(models)))

## make base regression object
forest = RandomForestClassifier(n_estimators = 100, max_depth = 6, random_state = 212)

## loop through all splits
i = 0
for train_index, test_index in skfold.split(x_train,y_train):
    ## get train and holdout sets
    x_train_train = x_train.iloc[train_index]
    x_holdout = x_train.iloc[test_index]
    y_train_train = y_train.iloc[train_index]
    y_holdout = y_train.iloc[test_index]

    ## loop through all models
    j = 0
    for model in models:
        if model == "baseline":
            y_mean_proba = y_train_train.value_counts(normalize=True,sort=False).sort_index().array
            y_mean_proba_array = np.ones((len(y_holdout),5))
            y_mean = 3.0
            y_mean_array = y_mean*np.ones(len(y_holdout))
            for k in range(0,len(y_holdout)):
                y_mean_proba_array[k]=y_mean_proba
            cv_roc_auc_ovo_model[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovo')
            cv_roc_auc_ovr_model[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovr')
            cv_acc_score_model[i,j] = accuracy_score(y_holdout,y_mean_array)

        else:
            ## make clone
            forest_clone = clone(forest)
        
        
            ## fit clone
            forest_clone.fit(x_train_train[model],y_train_train)

            y_proba_predict = forest_clone.predict_proba(x_holdout[model])

            ## record mse
            cv_roc_auc_ovo_model[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovo')
            cv_roc_auc_ovr_model[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovr')
            cv_acc_score_model[i,j] = accuracy_score(y_holdout,forest_clone.predict(x_holdout[model]))
        j=j+1
    i=i+1  

In [ ]:
models


In [18]:
ovo_max = np.argmax(np.mean(cv_roc_auc_ovo_model,axis=0))
ovr_max = np.argmax(np.mean(cv_roc_auc_ovr_model,axis=0))
acc_max = np.argmax(np.mean(cv_acc_score_model,axis=0))

print("The Model with Max ROC AUC OVO is ",models[ovo_max])
print("The Model with Max ROC AUC OVR is ",models[ovr_max])
print("The Model with Max Accuracy is ",models[acc_max])
print("\nFor Model:",models[ovo_max],"\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[ovo_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovo_max],
      "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[ovr_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovr_max],
      "\nThe Accuracy score is ",np.mean(cv_acc_score_model,axis=0)[ovo_max]
    )

print("\nFor Model:",models[acc_max],"\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[acc_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovo_max],
      "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[acc_max],
      #"with standard deviation of",np.std(cv_roc_auc_ovo_model,axis=0)[ovr_max],
      "\nThe Accuracy score is ",np.mean(cv_acc_score_model,axis=0)[acc_max]
    )


The Model with Max ROC AUC OVO is  ['Insured', 'Num_Vehicles', 'Num_Drivers']
The Model with Max ROC AUC OVR is  ['Insured', 'Num_Vehicles', 'Num_Drivers']
The Model with Max Accuracy is  ['Insured', 'Num_Vehicles', 'Marital', 'Add_VD']

For Model: ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003

For Model: ['Insured', 'Num_Vehicles', 'Marital', 'Add_VD'] 
The OVO ROC score is  0.7533778181045463 
The OVR ROC score is  0.7522175687811853 
The Accuracy score is  0.42824999999999996


In [19]:
for i in range(0,len(models)):
    if np.mean(cv_roc_auc_ovo_model,axis=0)[i] > 0.754 :
        print("For Model:",models[i],
              "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[i],
              "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[i],
              "\nThe Accuracy score is ",np.mean(cv_acc_score_model,axis=0)[i])

For Model: ['Insured', 'Num_Vehicles', 'Num_Drivers'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003
For Model: ['Insured', 'Num_Vehicles', 'Multi_VD'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003
For Model: ['Insured', 'Num_Drivers', 'Multi_VD'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003
For Model: ['Insured', 'Num_Vehicles', 'Num_Drivers', 'Multi_VD'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003
For Model: ['Insured', 'Num_Vehicles', 'Add_VD'] 
The OVO ROC score is  0.7544327492759777 
The OVR ROC score is  0.7534709250630878 
The Accuracy score is  0.42712500000000003
For Model: ['Insured', 'Num_Drivers', 'Add_VD'] 
The OVO ROC score is  0.7544327492759777 

In [ ]:
for i in range(0,len(models)):
    print("For Model:",models[i],"\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_model,axis=0)[i],
          "with standard deviation of ",np.std(cv_roc_auc_ovo_model,axis=0)[i],
          "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_model,axis=0)[i],
          "with standard deviation of ",np.std(cv_roc_auc_ovr_model,axis=0)[i],
)


In [32]:
## get all models we're interested in
model = ["Insured", "Num_Vehicles", "Num_Drivers"]

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state = 5309)
max_estimators = 500

## make empty mse holder
cv_roc_auc_ovo_estimators = np.zeros((5, 10))
cv_roc_auc_ovr_estimators = np.zeros((5, 10))
cv_acc_score_estimators = np.zeros((5, 10))

## make base regression object

## loop through all splits
i = 0
for train_index, test_index in skfold.split(x_train,y_train):
    ## get train and holdout sets
    x_train_train = x_train.iloc[train_index]
    x_holdout = x_train.iloc[test_index]
    y_train_train = y_train.iloc[train_index]
    y_holdout = y_train.iloc[test_index]

    ## loop through all models
    for j in range(1,10):
        forest = RandomForestClassifier(n_estimators = j*50, max_depth = 6, random_state = 212)
        
        if model == "baseline":
            y_mean_proba = y_train_train.value_counts(normalize=True,sort=False).sort_index().array
            y_mean_proba_array = np.ones((len(y_holdout),5))
            y_mean = 3.0
            y_mean_array = y_mean*np.ones(len(y_holdout))
            for k in range(0,len(y_holdout)):
                y_mean_proba_array[k]=y_mean_proba
            cv_roc_auc_ovo_estimators[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovo')
            cv_roc_auc_ovr_estimators[i,j] = roc_auc_score(y_holdout,y_mean_proba_array,multi_class='ovr')
            cv_acc_score_estimators[i,j] = accuracy_score(y_holdout,y_mean_array)

        else:
            ## make clone
            forest_clone = clone(forest)
        
            ## fit clone
            forest_clone.fit(x_train_train[model],y_train_train)

            y_proba_predict = forest_clone.predict_proba(x_holdout[model])

            ## record mse
            cv_roc_auc_ovo_estimators[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovo')
            cv_roc_auc_ovr_estimators[i,j] = roc_auc_score(y_holdout,y_proba_predict,multi_class='ovr')
            cv_acc_score_estimators[i,j] = accuracy_score(y_holdout,forest_clone.predict(x_holdout[model]))
    i=i+1  

In [33]:
for i in range(1,10):
    #if np.mean(cv_roc_auc_ovo_model,axis=0)[i] > 0.754 :
        print("For Number Estimators:",i*50,
              "\nThe OVO ROC score is ",np.mean(cv_roc_auc_ovo_estimators,axis=0)[i],
              "\nThe OVR ROC score is ",np.mean(cv_roc_auc_ovr_estimators,axis=0)[i],
              "\nThe Accuracy score is ",np.mean(cv_acc_score_estimators,axis=0)[i])

For Number Estimators: 50 
The OVO ROC score is  0.7544837106966362 
The OVR ROC score is  0.7535786285495851 
The Accuracy score is  0.4285
For Number Estimators: 100 
The OVO ROC score is  0.754318438698841 
The OVR ROC score is  0.7533723891479969 
The Accuracy score is  0.43
For Number Estimators: 150 
The OVO ROC score is  0.7544094293944374 
The OVR ROC score is  0.7535604884594262 
The Accuracy score is  0.43
For Number Estimators: 200 
The OVO ROC score is  0.75436614942987 
The OVR ROC score is  0.7535316348746828 
The Accuracy score is  0.42824999999999996
For Number Estimators: 250 
The OVO ROC score is  0.7544566997391956 
The OVR ROC score is  0.7535926350394233 
The Accuracy score is  0.42824999999999996
For Number Estimators: 300 
The OVO ROC score is  0.7544239027982439 
The OVR ROC score is  0.7535250744395512 
The Accuracy score is  0.42824999999999996
For Number Estimators: 350 
The OVO ROC score is  0.7545437332594165 
The OVR ROC score is  0.7536443508898744 
The A

In [34]:
np.mean(cv_roc_auc_ovo_estimators,axis=0)

array([0.        , 0.75448371, 0.75431844, 0.75440943, 0.75436615,
       0.7544567 , 0.7544239 , 0.75454373, 0.75444824, 0.75440513])